In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
from scipy.stats import distributions as iid
from scipy.optimize import minimize
from scipy.special import factorial2
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline


##############################################
#                Functions
##############################################

def dgp(N, mu, sigma, dist = 'normal'):
    """Generate a series of X observations from a type dist distribution.

    Satisfies model:
        X ~iid Normal(mu, sigma)

    Each element of the series is a single observation of X

    Inputs include
    - N :: number of observations to sample
    - mu :: Expectation of X
    - sigma :: Variance of X
    - dist :: one of ['normal', 'cauchy', 'poisson']
    """
    if dist=='normal':
        return iid.norm(loc=mu, scale=sigma).rvs(size=(N,1))
    if dist=='cauchy':
        return iid.cauchy(loc=mu, scale=sigma).rvs(size=(N,1))
    if dist=='poisson':
        return iid.poisson(mu).rvs(size=(N,1))

    
def moment_restriction(b, x, k):
    """k^th moment condition evaluated at the observation x
    
    note: k%2 = "k modulo 2" = remainder of k/2
          so k%2=0 if k is even
          (k%2==0) is an indicator function (=1 if k%2=0, 0 otherwise)
    """
    mu, sigma = b
    return (x - mu)**k - (k%2==0)*sigma**k*factorial2(k-1)
    

def gj(b,K,x):
    """The observation-level moment restriction for normal distribution null hypothesis.
    Observations of g_j(b). gj is a single observation
    
    K = # of moments to consider

    This defines the deviations from the predictions of our model; i.e.,
    k=1: e1 = x - mu,               where E[e1] = E[x - mu] = 0 under the null
    k=2: e2 = (x - mu)^2 - sigma^2, where E[e2] = E[(x-mu)^2 - sigma^2] = 0 under the null
    ...
    
    """
    gj = moment_restriction(b, x, np.arange(1,K+1))
    return gj


def gN(b,K,data):
    """Averages of g_j(b) over all data.
    """
    e = gj(b,K,data)

    # Check to see more obs. than moments.
    assert e.shape[0] > e.shape[1]
    
    # Return sample average of gj(b)'s
    gN = e.mean(axis=0)
    return gN


def Omegahat(b,K,data):
    """Estimate of the covariance matrix E[gN'gN]"""
    e = gj(b,K,data)
    # Recenter! We have Eu=0 under null.
    # Important to use this information.
    e = e - e.mean(axis=0) 
    return e.T@e/e.shape[0]


def J(b,K,W,data):
    m = gN(b,K,data) # Sample moments @ b
    N = len(data)
    # Criterion Function: N * (gN)^T * W * gN
    return N*m.T@W@m # Scale by sample size


def cu_gmm(data, K):
    """Continuously-Updated GMM
    In each step of the minimization, the covariance matrix estimate is updated
    with the new parameter value
    
    The lambda function creates a function J_b(b)
    where J_b(b) = J(b, K, W(b,K,data) data)
    
    The minimize function is used instead of the minimize_scalar because the
    minimize function can minimize over a vector of parameters instead
    of just a single parameter.
    """
    J_b = lambda b: J(b, K, inv(Omegahat(b,K,data)), data)
    J_min = minimize(J_b, x0=(mu,sigma))
    return J_min


def plots(type_list, proportions_dict, D=1000, title_size = 20, fig_size=(8,5)):
    """Plot the proportion of simulations for which the null hypothesis is rejected,
    depending on the number of moment conditions that were included in gj.
    
    A plot is created for each distribution in type_list. 
    
    type_list :: list of strings with names of distributions
    proportions_dict :: dictionary, each key is a distribution name, and value is the
        dataset of proportion of MC draws where the null hypothesis was rejected.
    """    
    # Create figure
    f, ax = plt.subplots(1, 1, figsize=fig_size)
    
    # For each distribution, create a plot of rejections vs moments included
    for dist in type_list:
        # Get data from dictionary
        y = np.array(proportions_dict[dist])*100 # convert portion to %
        x = proportions_dict['moments']
        # Generate plot of % rejected vs # of moments included in gj
        ax.plot(x, y, label=dist)
        ax.set_xlabel('number of moment conditions (K)')
        ax.set_ylabel(r'% of MC simulations that led to rejected H$_0$')
        
    # Add title
    ax.set_title(f'% of {D} Monte Carlo draws from distributions\nthat led to a rejected null hypothesis', 
                    fontsize=15)
    # Better x-tick marks (on even ticks)
    plt.xticks(range(2,max(x)+1,2))
    plt.grid(True)
    # Add legend
    ax.legend(shadow=True, fancybox=True)


    
##############################################
#                Main
##############################################
    
# True parameters
mu = 5
sigma = 3

# number of observations to pull
N = 10000
# Monte Carlo draws
D = 10
# Max. number of moment conditions to test
k_max = 50
# Significance level
alpha = 0.05

# Run some Monte Carlo!
# Try pulling data from different distributions in type_list
# For each distribution type, try many different #s of moment conditions
# For each # of moment conditions, run a Monte Carlo set of simulations to
#     calculate the portion of the MC simulations that lead to a rejected 
#     null hypothesis that the data came from a Normal distribution
type_list = ['normal', 'cauchy', 'poisson']
reject_portions = {} 
for dist in type_list:
    print(f'Working on {dist} distribution')
    rejections = []
    # Try different numbers of moment conditions (2 - k_max)
    k_range = trange(2, k_max+1) # range of # of moment conditions to use
    for K in k_range:
        P_values = []
        # Monte Carlo: simulate data D times
        for d in range(D):
            # Update status bar
            k_range.set_description(f'Moments = {K}/{k_max}, MC draw = {d}/{D}')
            # added a try-except loop because sometimes
            # the data leads to a singular covariance matrix
            soltn = None
            while soltn is None:
                try: 
                    # for each MC draw of data, get the GMM Solution
                    soltn = cu_gmm(dgp(N, mu, sigma, dist = dist), K)
                    # Minimized criterion function value
                    J_min = soltn.fun
                    # Calculate the prob. of observing this J value or greater if
                    # the true distribution is normal (assym. J = 0)
                    # p-value = 1 - CDF(J-value)
                    P_values.append(1 - iid.chi2.cdf(J_min, K-2, loc=0, scale=1))
                except TypeError:
                    print('TypeError: you probably overwrote the name of a function')
                except:
                    pass

        # Calculate the portion of MC draws where we would reject the null
        reject_portion = sum(1 for i in P_values if i < alpha) / D
        rejections.append(reject_portion)
    # Add proportions of rejections to dictionary for this sampling distribution
    reject_portions[dist] = rejections


reject_portions['moments'] = [k for k in range(2, k_max+1)]

# print table of proportions of MC draws that lead to a rejected null hypothesis (of normality)
print(pd.DataFrame(reject_portions, index=reject_portions['moments']))
        
        
# How does the proportion of rejected MC draws change with the number of moment conditions?
# For each of the distributions tested, we can see how the number of moment conditions used
#    changes how often we reject the null hypothesis. Ideally, we reject the null very
#    little if we are pulling from a normal distribution, and we reject the null very
#    often if we are pulling from a non-normal distribution. 
# For a given # of moments used, we have calculated the portion of the MC simulations that 
#    led to a rejected hypotheis. So let's plot (% MC simulations rejected) vs (# of moments used).
plots(type_list, reject_portions, D=D)



Working on normal distribution
